In [1]:
!pip install --quiet img2vec_pytorch
print('pip installed img2vec')

pip installed img2vec


In [2]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning) # quiet a plotly issue
filterwarnings(action='ignore', category=UserWarning) # quiet an img2vec issue

In [3]:
from img2vec_pytorch import Img2Vec
from PIL import Image
from arrow import now
from glob import glob
import pandas as pd
from os.path import basename

# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

# we're going to just read a few pictures while we're building
def get_from_glob(arg: str, tag: str, stop: int) -> list:
    time_get = now()
    result = []
    for index, input_file in enumerate(glob(pathname=arg)):
        if index < stop:
            name = basename(input_file)
            try:
                with Image.open(fp=input_file, mode='r') as image:
                    vector = img2vec.get_vec(image, tensor=True).numpy().reshape(512,)
                    result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
            except RuntimeError:
                # we only have a few failures so we're just going to discard them
                print('runtime failure: {}'.format(tag, name))
                pass
    print('encoded {} data {} rows in {}'.format(tag, len(result), now() - time_get))
    return result

STOP = 2500

img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=512)

time_start = now()
train = {basename(folder) : folder + '/*.jpg' for folder in glob('/kaggle/input/pakistan-currency-dataset/Pakistan/Training/*')}
train_data = [get_from_glob(arg=value, tag=key, stop=STOP) for key, value in train.items()]
df = pd.DataFrame(data=flatten(arg=train_data))
    
print('done in {}'.format(now() - time_start))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


encoded 10Rs data 385 rows in 0:00:33.324767
encoded 1000Rsback data 356 rows in 0:00:32.683930
encoded 50Rs data 355 rows in 0:00:34.942412
encoded 5000Rs data 376 rows in 0:00:39.034067
encoded 10Rsback data 399 rows in 0:00:33.004769
encoded 500Rsback data 377 rows in 0:00:31.378090
encoded 20Rs data 367 rows in 0:00:42.615202
encoded 100Rs data 393 rows in 0:00:31.051309
encoded 5000Rsback data 381 rows in 0:00:38.618905
encoded 1000Rs data 347 rows in 0:00:30.819196
encoded 50Rsback data 369 rows in 0:00:35.221382
encoded 500Rs data 378 rows in 0:00:30.338298
encoded 100Rsback data 399 rows in 0:00:31.911942
encoded others data 612 rows in 0:01:39.997315
encoded 20Rsback data 378 rows in 0:00:39.100549
done in 0:09:44.654813


In [4]:
from plotly.express import histogram
histogram(data_frame=df, x='tag')

Our classes are unbalanced but not severely so.

In [5]:
from arrow import now
from umap import UMAP

time_start = now()
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=500)
plot_df = pd.concat(objs=[df, pd.DataFrame(data=umap.fit_transform(X=df['value'].apply(pd.Series)), columns=['x', 'y'])], axis=1)
print('done with UMAP in {}'.format(now() - time_start))

2024-02-22 18:12:09.448618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 18:12:09.448823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 18:12:09.613620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=500, n_jobs=1, random_state=2024, verbose=True)
Thu Feb 22 18:12:25 2024 Construct fuzzy simplicial set
Thu Feb 22 18:12:25 2024 Finding Nearest Neighbors
Thu Feb 22 18:12:25 2024 Building RP forest with 9 trees
Thu Feb 22 18:12:31 2024 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	Stopping threshold met -- exiting after 6 iterations
Thu Feb 22 18:12:52 2024 Finished Nearest Neighbor Search
Thu Feb 22 18:12:56 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Thu Feb 22 18:13:07 2024 Finished embedding
done with UMAP in 0:00:43.600496


In [6]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='x', y='y', color='tag', hover_name='name', height=900).show()

This is not encourating; UMAP clusters our pictures, but not by tag. Let's be hopeful and build a model.

In [7]:
from arrow import now

# we don't have test data so let's use the validation set as our test data
test = {basename(folder) : folder + '/*.jpg' for folder in glob('/kaggle/input/pakistan-currency-dataset/Pakistan/Valid/*')}

time_start = now()
test_data = [get_from_glob(arg=value, tag=key, stop=STOP) for key, value in test.items()]
test_df = pd.DataFrame(data=flatten(arg=test_data))
print('done in {}'.format(now() - time_start))

encoded 10Rs data 9 rows in 0:00:01.141121
encoded 1000Rsback data 9 rows in 0:00:01.121860
encoded 50Rs data 10 rows in 0:00:01.892055
encoded 5000Rs data 10 rows in 0:00:01.175533
encoded 10Rsback data 8 rows in 0:00:01.474214
encoded 500Rsback data 10 rows in 0:00:00.771203
encoded 20Rs data 10 rows in 0:00:02.431633
encoded 100Rs data 10 rows in 0:00:01.402578
encoded 5000Rsback data 10 rows in 0:00:00.934374
encoded 1000Rs data 9 rows in 0:00:01.584139
encoded 50Rsback data 8 rows in 0:00:01.726244
encoded 500Rs data 10 rows in 0:00:00.775512
encoded 100Rsback data 10 rows in 0:00:02.147126
encoded others data 52 rows in 0:00:08.818400
encoded 20Rsback data 10 rows in 0:00:01.829472
done in 0:00:29.264244


In [8]:
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from arrow import now

best_k = 1
best = 0
# let's step through a range of cluster sizes to find the one that will give us the best accuracy
for n_neighbors in range(2, 15):
    current = KNeighborsClassifier(n_neighbors=n_neighbors)
    current.fit(X=df['value'].apply(pd.Series), y=df['tag'])
    score = f1_score(average='weighted', labels=test_df['tag'].unique().tolist(), y_true=test_df['tag'], y_pred=current.predict(X=test_df['value'].apply(pd.Series)))
    if score > best:
        best = score
        best_k = n_neighbors
    print('neighbors: {} score: {:5.4f}'.format(n_neighbors, score))
        
time_start = now()
print('building best-k model with k = {}'.format(best_k))
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X=df['value'].apply(pd.Series), y=df['tag'])
print(classification_report(labels=test_df['tag'].unique().tolist(), y_true=test_df['tag'], y_pred=knn.predict(X=test_df['value'].apply(pd.Series))))
print('model time: {}'.format(now() - time_start))

neighbors: 2 score: 0.6054
neighbors: 3 score: 0.6832
neighbors: 4 score: 0.6730
neighbors: 5 score: 0.6839
neighbors: 6 score: 0.6771
neighbors: 7 score: 0.6832
neighbors: 8 score: 0.6819
neighbors: 9 score: 0.6838
neighbors: 10 score: 0.6814
neighbors: 11 score: 0.6644
neighbors: 12 score: 0.6725
neighbors: 13 score: 0.6512
neighbors: 14 score: 0.6467
building best-k model with k = 5
              precision    recall  f1-score   support

        10Rs       0.43      0.67      0.52         9
  1000Rsback       0.42      0.89      0.57         9
        50Rs       0.71      0.50      0.59        10
      5000Rs       0.60      0.90      0.72        10
    10Rsback       0.57      0.50      0.53         8
   500Rsback       0.43      0.30      0.35        10
        20Rs       0.67      0.60      0.63        10
       100Rs       0.25      0.20      0.22        10
  5000Rsback       1.00      0.30      0.46        10
      1000Rs       0.53      1.00      0.69         9
    50Rsback    